## Import

In [3]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
import time
import schedule as sc
from datetime import datetime as dt

## Fake UA

In [4]:
FakeUserAgen = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2919.83 Safari/537.36',
}

In [5]:
# Bypass Error 403
# FakeUserAgen = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
# }

## Scraping Serambi

In [10]:
def scraping_serambi(tgl_awal, tgl_akhir, bln, thn):
    titles = []
    links = []
    dates = []
    contents = []
    labels = []
    with tqdm(total=tgl_akhir-tgl_awal+1, desc="Processing", unit="iter") as pbar:
        while tgl_awal <= tgl_akhir: #iterasi berita tiap hari 
            halaman = 1 #halaman def
            ttl_hal = 1 #total halaman def
            while halaman <= ttl_hal: #iterasi tiap halaman 
                #data retrieval
                url = f'https://aceh.tribunnews.com/index-news/nanggroe?date={thn}-{bln}-{tgl_awal}&page={halaman}'
                html = requests.get(url, headers=FakeUserAgen).text
                bsobj = soup(html,'lxml')
                data = bsobj.select('ul.lsi li.ptb15') #list berita
                jmlh_hal = bsobj.find('div',{'class':'pt10 pb10'}) #jumlah berita 
                ttl_hal = math.ceil(int(jmlh_hal.find('b').text)/20) #jumalah halaman (jumlah berita/20 ceil) 
                for i in range(len(data)): #iterasi list berita
                    try:
                        html_cntn = requests.get(data[i].find('h3').find('a').get('href'), headers=FakeUserAgen).text
                        bsobj_cntn = soup(html_cntn,'lxml')
                        isi = bsobj_cntn.find('div',class_='side-article txt-article multi-fontsize editcontent')
                        contents.append(isi.text)

                        label = data[i].find('h4')
                        labels.append(label.text if label is not None else 'null')
                        titles.append(data[i].find('h3').find('a').get('title')) #get headlines
                        links.append(data[i].find('h3').find('a').get('href')) #get links
                        dates.append(f'{thn}-{bln}-{tgl_awal}') #get times
                    except AttributeError:
                        pass
                    # print(i, end=' ')
                    if i == 19:
                        break #ganti halaman setelah berita ke-20
                # print(f'|{halaman} done')
                halaman += 1 #ganti halaman
            pbar.update(1)
            tgl_awal += 1 #ganti tanggal
    # simpan kedalam dataframe
    df = pd.DataFrame({
    'date': dates,
    'headline': titles,
    'link': links,
    'content': contents,
    'label' : labels
    })

    df.to_csv(f'Serambi_{bln}-{thn}.csv', index=False, encoding='utf-8')
    
    return df

## Scraping

In [11]:
#gunakan method scraping_serambi disini ...
scraping_serambi(1, 30, 6, 2024)

Processing: 100%|█████████████████████████████| 30/30 [00:09<00:00,  3.03iter/s]


,date,headline,link,content,label


## Scraping otomatis

In [105]:
# def Scraping():
#     scraping_serambi(dt.now().date().day,dt.now().date().day,dt.now().date().month,dt.now().date().year)

In [104]:
# sc.every().day.at('15:25').do(Scraping)
# while True:
#     sc.run_pending()
#     time.sleep(1)